In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import datetime
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions
from pycontrails.core.met import MetDataArray, MetDataset
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)
from boxm import BoxModel

import os
import psutil

In [2]:
# PHOTOLYSIS PARAMETERS IN FORMAT J = L*COSX**M*EXP(-N*SECX)
consts = pd.read_pickle('J.pkl')
    
# Extract the constants
photol_idx, L, M, N = np.array(consts).T

In [3]:
# Initialise coord arrays
longitude = np.arange(-180, 180, 5)
latitude = np.arange(-90, 90, 5)
level = np.array([1013, 912, 810, 709, 607, 505, 404, 302, 201, 100])
time = [datetime.datetime(2000, 6, 1, h, 0) for h in range(0, 10)] # always GMT!
photol_params = photol_idx
photol_coeffs = np.arange(1, 96 + 1) # from Fortran indexing (1 to 96)
therm_coeffs = np.arange(1, 510 + 1) # from Fortran indexing (1 to 510)
species = [ 'O1D', 'O', 'OH', 'NO2', 'NO3', 'O3', 'N2O5', 'NO', 'HO2', 'H2', 'CO', 
           'H2O2', 'HONO', 'HNO3', 'HO2NO2', 'SO2', 'SO3', 'HSO3', 'NA', 'SA', 
           'CH4', 'CH3O2', 'C2H6', 'C2H5O2', 'C3H8', 'IC3H7O2', 'RN10O2', 'NC4H10', 
           'RN13O2', 'C2H4', 'HOCH2CH2O2', 'C3H6', 'RN9O2', 'TBUT2ENE', 'RN12O2', 
           'NRN6O2', 'NRN9O2', 'NRN12O2', 'HCHO', 'HCOOH', 'CH3CO2H', 'CH3CHO', 
           'C5H8', 'RU14O2', 'NRU14O2', 'UCARB10', 'APINENE', 'RTN28O2', 'NRTN28O2', 
           'RTN26O2', 'TNCARB26', 'RCOOH25', 'BPINENE', 'RTX28O2', 'NRTX28O2', 
           'RTX24O2', 'TXCARB24', 'TXCARB22', 'C2H2', 'CARB3', 'BENZENE', 'RA13O2', 
           'AROH14', 'TOLUENE', 'RA16O2', 'AROH17', 'OXYL', 'RA19AO2', 'RA19CO2', 
           'CH3CO3', 'C2H5CHO', 'C2H5CO3', 'CH3COCH3', 'RN8O2', 'RN11O2', 'CH3OH', 
           'C2H5OH', 'NPROPOL', 'IPROPOL', 'CH3CL', 'CH2CL2', 'CHCL3', 'CH3CCL3', 
           'TCE', 'TRICLETH', 'CDICLETH', 'TDICLETH', 'CARB11A', 'RN16O2', 'RN15AO2', 
           'RN19O2', 'RN18AO2', 'RN13AO2', 'RN16AO2', 'RN15O2', 'UDCARB8', 'UDCARB11', 
           'CARB6', 'UDCARB14', 'CARB9', 'MEK', 'HOCH2CHO', 'RN18O2', 'CARB13', 
           'CARB16', 'HOCH2CO3', 'RN14O2', 'RN17O2', 'UCARB12', 'RU12O2', 'CARB7', 
           'RU10O2', 'NUCARB12', 'NRU12O2', 'NOA', 'RTN25O2', 'RTN24O2', 'RTN23O2', 
           'RTN14O2', 'TNCARB10', 'RTN10O2', 'RTX22O2', 'CH3NO3', 'C2H5NO3', 'RN10NO3', 
           'IC3H7NO3', 'RN13NO3', 'RN16NO3', 'RN19NO3', 'HOC2H4NO3', 'RN9NO3', 'RN12NO3', 
           'RN15NO3', 'RN18NO3', 'RU14NO3', 'RA13NO3', 'RA16NO3', 'RA19NO3', 'RTN28NO3', 
           'RTN25NO3', 'RTX28NO3', 'RTX24NO3', 'RTX22NO3', 'CH3OOH', 'C2H5OOH', 'RN10OOH', 
           'IC3H7OOH', 'RN13OOH', 'RN16OOH', 'RN19OOH', 'RA13OOH', 'RA16OOH', 'RA19OOH', 
           'HOC2H4OOH', 'RN9OOH', 'RN12OOH', 'RN15OOH', 'RN18OOH', 'CH3CO3H', 'C2H5CO3H', 
           'HOCH2CO3H', 'RN8OOH', 'RN11OOH', 'RN14OOH', 'RN17OOH', 'RU14OOH', 'RU12OOH', 
           'RU10OOH', 'NRN6OOH', 'NRN9OOH', 'NRN12OOH', 'NRU14OOH', 'NRU12OOH', 'RTN28OOH', 
           'NRTN28OOH', 'RTN26OOH', 'RTN25OOH', 'RTN24OOH', 'RTN23OOH', 'RTN14OOH', 
           'RTN10OOH', 'RTX28OOH', 'RTX24OOH', 'RTX22OOH', 'NRTX28OOH', 'CARB14', 'CARB17', 
           'CARB10', 'CARB12', 'CARB15', 'CCARB12', 'ANHY', 'TNCARB15', 'RAROH14', 'ARNOH14', 
           'RAROH17', 'ARNOH17', 'PAN', 'PPN', 'PHAN', 'RU12PAN', 'MPAN', 'RTN26PAN', 'P2604', 
           'P4608', 'P2631', 'P2635', 'P4610', 'P2605', 'P2630', 'P2629', 'P2632', 'P2637', 
           'P3612', 'P3613', 'P3442', 'CH3O2NO2', 'EMPOA', 'P2007']

In [4]:
# Import met data from ERA5
era5 = ERA5(
        time=(time[0], time[-1]),
        variables=[
                "t",
                "q",
                "u",
                "v",
                "w",
                "z",
                "relative_humidity"
        ],
        pressure_levels=[1000, 925, 800, 700, 600, 500, 400, 300, 200, 100] 
)

In [5]:
# download data from ERA5 (or open from cache)
met = era5.open_metdataset()
met.data = met.data.transpose("latitude", "longitude", "level", "time", ...)

met

MetDataset with data:

<xarray.Dataset>
Dimensions:                              (latitude: 721, longitude: 1440,
                                          level: 10, time: 10)
Coordinates:
  * longitude                            (longitude) float64 -180.0 ... 179.8
  * latitude                             (latitude) float64 -90.0 ... 90.0
  * level                                (level) float64 100.0 200.0 ... 1e+03
  * time                                 (time) datetime64[ns] 2000-06-01 ......
    air_pressure                         (level) float32 1e+04 2e+04 ... 1e+05
    altitude                             (level) float32 1.618e+04 ... 110.9
Data variables:
    air_temperature                      (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    specific_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    eastward_wind                        (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    northward_wind                       (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    lagrangian_tendency_of_air_pressure  (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    geopotential                         (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
    relative_humidity                    (latitude, longitude, level, time) float32 dask.array<chunksize=(1, 1440, 10, 1), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.6
    history:              2023-08-09 17:34:07 GMT by grib_to_netcdf-2.25.1: /...
    pycontrails_version:  0.42.2
    met_source:           ERA5

In [6]:
met.shape

(1440, 721, 10, 10)

In [7]:
# initialise example chem MetDataset
chem = xr.Dataset(
    {
        
        "local_time": (["latitude", "longitude", "time"],
                np.zeros((len(latitude), len(longitude), len(time)))),
        "sza": (["latitude", "longitude", "time"], 
                np.zeros((len(latitude), len(longitude), len(time)))),
        "J": (["latitude", "longitude", "time", "photol_params"], 
                np.zeros((len(latitude), len(longitude), len(time), len(photol_params)))),
        "DJ": (["latitude", "longitude", "level", "time", "photol_coeffs"], 
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(photol_coeffs)))),
        "RC": (["latitude", "longitude", "level", "time", "therm_coeffs"],
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(therm_coeffs)))),
        "soa": (["latitude", "longitude", "level", "time"],
                np.zeros((len(latitude), len(longitude), len(level), len(time)))),
        "mom": (["latitude", "longitude", "level", "time"],
                np.zeros((len(latitude), len(longitude), len(level), len(time)))),
        "Y": (["latitude", "longitude", "level", "time", "species"], 
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(species)))),
        "EM": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(species)))),
        "FL": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(species)))),

    },
    coords={
        "latitude": latitude,
        "longitude": longitude, 
        "level": level,
        "time": time,
        "photol_params": photol_params,
        "photol_coeffs": photol_coeffs,
        "therm_coeffs": therm_coeffs,
        "species": species,
    }
).chunk({"time": 1})
 
chem = MetDataset(chem)
chem.data = chem.data.transpose("latitude", "longitude", "level", "time", ...)

: 

In [ ]:
chem

MetDataset with data:

<xarray.Dataset>
Dimensions:        (latitude: 36, longitude: 72, time: 10, photol_params: 34,
                    level: 10, photol_coeffs: 96, therm_coeffs: 510,
                    species: 219)
Coordinates:
  * latitude       (latitude) float64 -90.0 -85.0 -80.0 -75.0 ... 75.0 80.0 85.0
  * longitude      (longitude) float64 -180.0 -175.0 -170.0 ... 170.0 175.0
  * level          (level) float64 100.0 201.0 302.0 ... 810.0 912.0 1.013e+03
  * time           (time) datetime64[ns] 2000-06-01 ... 2000-06-01T09:00:00
  * photol_params  (photol_params) float64 1.0 2.0 3.0 4.0 ... 55.0 56.0 57.0
  * photol_coeffs  (photol_coeffs) int64 1 2 3 4 5 6 7 ... 90 91 92 93 94 95 96
  * therm_coeffs   (therm_coeffs) int64 1 2 3 4 5 6 ... 505 506 507 508 509 510
  * species        (species) <U10 'O1D' 'O' 'OH' ... 'CH3O2NO2' 'EMPOA' 'P2007'
    air_pressure   (level) float64 1e+04 2.01e+04 ... 9.12e+04 1.013e+05
    altitude       (level) float64 1.618e+04 1.175e+04 9.119e+03 ... 879.1 2.081
Data variables:
    local_time     (latitude, longitude, time) float64 dask.array<chunksize=(36, 72, 1), meta=np.ndarray>
    sza            (latitude, longitude, time) float64 dask.array<chunksize=(36, 72, 1), meta=np.ndarray>
    J              (latitude, longitude, time, photol_params) float64 dask.array<chunksize=(36, 72, 1, 34), meta=np.ndarray>
    DJ             (latitude, longitude, level, time, photol_coeffs) float64 dask.array<chunksize=(36, 72, 10, 1, 96), meta=np.ndarray>
    RC             (latitude, longitude, level, time, therm_coeffs) float64 dask.array<chunksize=(36, 72, 10, 1, 510), meta=np.ndarray>
    soa            (latitude, longitude, level, time) float64 dask.array<chunksize=(36, 72, 10, 1), meta=np.ndarray>
    mom            (latitude, longitude, level, time) float64 dask.array<chunksize=(36, 72, 10, 1), meta=np.ndarray>
    Y              (latitude, longitude, level, time, species) float64 dask.array<chunksize=(36, 72, 10, 1, 219), meta=np.ndarray>
    EM             (latitude, longitude, level, time, species) float64 dask.array<chunksize=(36, 72, 10, 1, 219), meta=np.ndarray>
    FL             (latitude, longitude, level, time, species) float64 dask.array<chunksize=(36, 72, 10, 1, 219), meta=np.ndarray>

In [ ]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

328540160


In [ ]:
chem["Y"].shape

(36, 72, 10, 10, 219)

In [ ]:
for s in species:
    chem["Y"].data.loc[:, :, 201, time[0], s] = np.loadtxt("species/" + s + "_MONTH_1_LEVEL_8.csv",
                                                           delimiter=",")

FileNotFoundError: species/NA_MONTH_1_LEVEL_8.csv not found.

In [ ]:
boxm = BoxModel(met, chem)

In [ ]:
boxm.met

In [ ]:
boxm.chem

In [ ]:
emi = xr.Dataset(
    {
        "EM": (["latitude", "longitude", "level", "time", "species"],
                np.zeros((len(latitude), len(longitude), len(level), len(time), len(species)))),
    },
    coords={
        "latitude": latitude,
        "longitude": longitude, 
        "level": level,
        "time": time,
        "species": species,
    }
)

In [ ]:
emi = MetDataset(emi)
emi = emi.data.transpose("latitude", "longitude", "level", "time", ...)

In [ ]:
emi

In [ ]:
boxm.params

In [ ]:
boxm.eval(source=emi)